#Pemrograman Tugas Akhir:


"Segmentasi Burned Area menggunakan Model U-Net pada Citra Landsat 9 (Studi Kasus: Sumatera Selatan)"


*   Data Preprocessing
    * Koreksi Atmosferik (Dark Object Substraction)
    * Cloud Masking (QA Bands)
    * Komposit Citra (False Color)
*   Ekstraksi Fitur
    * NDVI
    * NBR
    * dNBR
*   Band Stacking
*   Masking Citra



Tools & Platform

![Google Earth Engine](https://img.shields.io/badge/-GEE-34A853?style=flat&logo=google-earth&logoColor=white)  ![Pandas](https://img.shields.io/badge/-Pandas-150458?style=flat&logo=pandas&logoColor=white)  ![Folium](https://img.shields.io/badge/-Folium-77B829?style=flat&logo=leaflet&logoColor=white)  ![geemap](https://img.shields.io/badge/-geemap-34A853?style=flat&logo=googleearth&logoColor=white)



# Inisialisasi dan Authentikasi GEE

In [1]:
import ee
ee.Authenticate() #Authentikasi GEE
ee.Initialize(project='ee-analafeanalber9') #Inisialisasi GEE dengan ID Project GEE

# Data Description (Data Raw)

## Prefire (01-01-2023 until 31-07-2023)

In [2]:
import ee
import pandas as pd

#Inisiasi Pathrow dan rentang waktu data digunakan
path = 124
row = 62
start_date = '2023-01-01'
end_date = '2023-07-31'

#Memanggil Koleksi Citra berdasarkan pathrow dan rentang waktu data
landsat9_col = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', path)) \
    .filter(ee.Filter.eq('WRS_ROW', row)) \
    .filterDate(start_date, end_date)

#Memberikan informasi tanggal citra tersedia, persentas cloud cover serta id citra
dates = landsat9_col.aggregate_array('DATE_ACQUIRED').getInfo()
clouds = landsat9_col.aggregate_array('CLOUD_COVER').getInfo()
ids = landsat9_col.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()


#Fungsi membuat dataframe
if dates:
    df = pd.DataFrame({
        'Pathrow': 124062,
        'Kondisi': 'Prefire',
        'Tanggal Citra': dates,
        'Cloud Cover (%)': clouds,
        'Product ID': ids
    })
    df
else:
    print("Tidak ada citra tersedia pada rentang waktu dan path/row tersebut.")

#Menampilkan dataframe sort berdasarkan cloud cover persentase terkecil ke terbesar
df = df.sort_values(by='Cloud Cover (%)', ascending=True).reset_index(drop=True)
df


,Pathrow,Kondisi,Tanggal Citra,Cloud Cover (%),Product ID
0,124062,Prefire,2023-07-26,14.07,LC09_L1TP_124062_20230726_20230726_02_T1
1,124062,Prefire,2023-04-21,29.16,LC09_L1TP_124062_20230421_20230421_02_T1
2,124062,Prefire,2023-06-24,33.04,LC09_L1TP_124062_20230624_20230624_02_T1
3,124062,Prefire,2023-03-20,54.76,LC09_L1TP_124062_20230320_20230320_02_T1
4,124062,Prefire,2023-04-05,56.74,LC09_L1TP_124062_20230405_20230405_02_T1
5,124062,Prefire,2023-05-23,57.75,LC09_L1TP_124062_20230523_20230523_02_T1
6,124062,Prefire,2023-07-10,83.95,LC09_L1TP_124062_20230710_20230710_02_T1
7,124062,Prefire,2023-01-15,84.48,LC09_L1TP_124062_20230115_20230313_02_T1
8,124062,Prefire,2023-02-16,86.69,LC09_L1TP_124062_20230216_20230310_02_T1
9,124062,Prefire,2023-06-08,93.87,LC09_L1TP_124062_20230608_20230608_02_T1


In [3]:
import ee
import folium
import geemap.foliumap as geemap

#Fungsi folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# Filter Path/Row dan Tanggal
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'

#Akses koleksi citra
landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()

#metadata tanggal dan cloud cover
date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')

#Visualisasi Band
band5_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band7_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band4_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'red', 'darkred']
}


#Peta Interaktif
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(landsat9_toa.select('B7'), band7_vis_params, 'Band 7 (SWIR)')
m.add_ee_layer(landsat9_toa.select('B5'), band5_vis_params, 'Band 5 (NIR)')
m.add_ee_layer(landsat9_toa.select('B4'), band4_vis_params, 'Band 4 (RED)')

#Kontrol layer
m.add_child(folium.LayerControl())

#Tambahkan info di atas peta
title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Pathrow 124062 (Prefire)
     </b></h3>
     <h3 align="center" style="font-size:16px"><b>
     Tanggal Citra: {date_acquired} | Cloud Cover: {cloud_cover}%
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-07-26
Persentase Cloud Cover: 14.07%


In [4]:
import ee
import pandas as pd

path = 125
row = 62
start_date = '2023-01-01'
end_date = '2023-07-31'


landsat9_col = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', path)) \
    .filter(ee.Filter.eq('WRS_ROW', row)) \
    .filterDate(start_date, end_date)


dates = landsat9_col.aggregate_array('DATE_ACQUIRED').getInfo()
clouds = landsat9_col.aggregate_array('CLOUD_COVER').getInfo()
ids = landsat9_col.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()


if dates:
    df = pd.DataFrame({
        'Pathrow': 125062,
        'Kondisi': 'Prefire',
        'Tanggal Citra': dates,
        'Cloud Cover (%)': clouds,
        'Product ID': ids
    })
    df
else:
    print("Tidak ada citra tersedia pada rentang waktu dan path/row tersebut.")

df = df.sort_values(by='Cloud Cover (%)', ascending=True).reset_index(drop=True)
df


,Pathrow,Kondisi,Tanggal Citra,Cloud Cover (%),Product ID
0,125062,Prefire,2023-05-14,26.38,LC09_L1TP_125062_20230514_20230514_02_T1
1,125062,Prefire,2023-01-22,50.78,LC09_L1TP_125062_20230122_20230313_02_T1
2,125062,Prefire,2023-06-15,65.17,LC09_L1TP_125062_20230615_20230615_02_T1
3,125062,Prefire,2023-04-12,66.80,LC09_L1TP_125062_20230412_20230412_02_T1
4,125062,Prefire,2023-01-06,73.91,LC09_L1TP_125062_20230106_20230314_02_T1
5,125062,Prefire,2023-04-28,75.61,LC09_L1TP_125062_20230428_20230428_02_T1
6,125062,Prefire,2023-02-07,87.14,LC09_L1TP_125062_20230207_20230310_02_T1
7,125062,Prefire,2023-07-01,90.50,LC09_L1TP_125062_20230701_20230701_02_T1
8,125062,Prefire,2023-05-30,91.37,LC09_L1TP_125062_20230530_20230530_02_T1
9,125062,Prefire,2023-03-27,92.76,LC09_L1TP_125062_20230327_20230327_02_T1


In [5]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

start_date = '2023-01-01'
end_date = '2023-07-31'

landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()

date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')

band5_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band7_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band4_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'red', 'darkred']
}


#Peta Interaktif
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(landsat9_toa.select('B7'), band7_vis_params, 'Band 7 (SWIR)')
m.add_ee_layer(landsat9_toa.select('B5'), band5_vis_params, 'Band 5 (NIR)')
m.add_ee_layer(landsat9_toa.select('B4'), band4_vis_params, 'Band 4 (RED)')
#Kontrol layer
m.add_child(folium.LayerControl())

#Tambahkan info di atas peta
title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Pathrow 125062 (Prefire)
     </b></h3>

     <h3 align="center" style="font-size:16px"><b>
     Tanggal Citra: {date_acquired} | Cloud Cover: {cloud_cover}%
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m

Citra yang ditampilkan diambil pada tanggal: 2023-05-14
Persentase Cloud Cover: 26.38%


## Postfire (01-11-2023 until 31-12-2023

In [6]:
import ee
import pandas as pd

path = 124
row = 62
start_date = '2023-11-01'
end_date = '2023-12-31'

landsat9_col = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', path)) \
    .filter(ee.Filter.eq('WRS_ROW', row)) \
    .filterDate(start_date, end_date)

dates = landsat9_col.aggregate_array('DATE_ACQUIRED').getInfo()
clouds = landsat9_col.aggregate_array('CLOUD_COVER').getInfo()
ids = landsat9_col.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

if dates:
    df = pd.DataFrame({
        'Pathrow': 124062,
        'Kondisi': 'Postfire',
        'Tanggal Citra': dates,
        'Cloud Cover (%)': clouds,
        'Product ID': ids
    })
    df
else:
    print("Tidak ada citra tersedia pada rentang waktu dan path/row tersebut.")

df = df.sort_values(by='Cloud Cover (%)', ascending=True).reset_index(drop=True)
df


,Pathrow,Kondisi,Tanggal Citra,Cloud Cover (%),Product ID
0,124062,Postfire,2023-11-15,23.63,LC09_L1TP_124062_20231115_20231115_02_T1
1,124062,Postfire,2023-12-17,69.01,LC09_L1TP_124062_20231217_20231217_02_T1


In [7]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

start_date = '2023-11-01'
end_date = '2023-12-31'


landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()


date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')


band5_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band7_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band4_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'red', 'darkred']
}


m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(landsat9_toa.select('B7'), band7_vis_params, 'Band 7 (SWIR)')
m.add_ee_layer(landsat9_toa.select('B5'), band5_vis_params, 'Band 5 (NIR)')
m.add_ee_layer(landsat9_toa.select('B4'), band4_vis_params, 'Band 4 (RED)')
m.add_child(folium.LayerControl())

title_html = f'''
<h3 align="center" style="font-size:20px"><b>
     Pathrow 124062 (Postfire)
     </b></h3>
     <h3 align="center" style="font-size:16px"><b>
     Tanggal Citra: {date_acquired} | Cloud Cover: {cloud_cover}%
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-11-15
Persentase Cloud Cover: 23.63%


In [8]:
import ee
import pandas as pd

path = 125
row = 62
start_date = '2023-11-01'
end_date = '2023-12-31'

landsat9_col = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', path)) \
    .filter(ee.Filter.eq('WRS_ROW', row)) \
    .filterDate(start_date, end_date)

dates = landsat9_col.aggregate_array('DATE_ACQUIRED').getInfo()
clouds = landsat9_col.aggregate_array('CLOUD_COVER').getInfo()
ids = landsat9_col.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

# Buat DataFrame
if dates:
    df = pd.DataFrame({
        'Pathrow': 125062,
        'Kondisi': 'Postfire',
        'Tanggal Citra': dates,
        'Cloud Cover (%)': clouds,
        'Product ID': ids
    })
    df
else:
    print("Tidak ada citra tersedia pada rentang waktu dan path/row tersebut.")

df


,Pathrow,Kondisi,Tanggal Citra,Cloud Cover (%),Product ID
0,125062,Postfire,2023-11-06,41.67,LC09_L1TP_125062_20231106_20231106_02_T1
1,125062,Postfire,2023-11-22,92.79,LC09_L1TP_125062_20231122_20231122_02_T1
2,125062,Postfire,2023-12-08,69.28,LC09_L1TP_125062_20231208_20231208_02_T1
3,125062,Postfire,2023-12-24,100.00,LC09_L1TP_125062_20231224_20231224_02_T1


In [9]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

start_date = '2023-11-01'
end_date = '2023-12-31'

landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()

date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')

band5_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band7_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'white']
}
band4_vis_params = {
    'min': 0.0,
    'max': 0.5,
    'palette': ['black', 'red', 'darkred']
}



m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(landsat9_toa.select('B7'), band7_vis_params, 'Band 7 (SWIR)')
m.add_ee_layer(landsat9_toa.select('B5'), band5_vis_params, 'Band 5 (NIR)')
m.add_ee_layer(landsat9_toa.select('B4'), band4_vis_params, 'Band 4 (RED)')
m.add_child(folium.LayerControl())

title_html = f'''
  <h3 align="center" style="font-size:20px"><b>
     Pathrow 125062 (Postfire)
     </b></h3>
     <h3 align="center" style="font-size:16px"><b>
     Tanggal Citra: {date_acquired} | Cloud Cover: {cloud_cover}%
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-11-06
Persentase Cloud Cover: 41.67%


# Data Preprocessing

## Koreksi Atmosferik

### Pathrow 124062 (Prefire)




In [10]:
import ee
import folium
import geemap.foliumap as geemap

#Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  #Mengambil URL peta dari objek citra EE dengan visualisasi tertentu

     #Menambahkan layer citra EE ke peta Folium menggunakan TileLayer
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format, # URL tile GEE
        attr='Google Earth Engine', #Atribut sumber tile
        name=name, #Nama layer untuk kontrol layer di peta
        overlay=True, #Menandakan ini layer tambahan, bukan base map
        control=True  #Memungkinkan kontrol layer aktif/nonaktif
    ).add_to(self) #Menambahkan layer ke map
folium.Map.add_ee_layer = add_ee_layer


# Filter Path/Row dan Tanggal
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'

#Ambil citra dari koleksi berdasarkan cloud cover terkecil
landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()

#Ambil metadata tanggal dan cloud cover
date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')

#DARK OBJECT SUBTRACTION (DOS)
def dark_object_subtraction(img, band_names): #memanggil band names di dark_object_substractiobn (B7,B5,B4)
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),  # mendekati nilai minimum
            geometry=band.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

#Terapkan DOS untuk band 7, 5, 4
dos_corrected = dark_object_subtraction(landsat9_toa, ['B7', 'B5', 'B4'])


false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}


m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'DOS False Color (B7-B5-B4)')
m.add_child(folium.LayerControl()) # Kontrol layer

#informasi tanggal dan cloud cover di atas peta
title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Koreksi Atmosferik (DOS) | Pathrow 124062 | Prefire
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-07-26
Persentase Cloud Cover: 14.07%


### Pathrow 125062 (Prefire)

In [11]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'


landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()


date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')


def dark_object_subtraction(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),  # mendekati nilai minimum
            geometry=band.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)

    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())


dos_corrected = dark_object_subtraction(landsat9_toa, ['B7', 'B5', 'B4'])


false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}


m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'DOS False Color (B7-B5-B4)')
m.add_child(folium.LayerControl())


title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Koreksi Atmosferik (DOS) | Pathrow 125062 | Prefire
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-05-14
Persentase Cloud Cover: 26.38%


### Pathrow 124062 (Postfire)

In [12]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()


date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')


def dark_object_subtraction(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),  # mendekati nilai minimum
            geometry=band.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

dos_corrected = dark_object_subtraction(landsat9_toa, ['B7', 'B5', 'B4'])

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}


m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'DOS False Color (B7-B5-B4)')
m.add_child(folium.LayerControl())


title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Koreksi Atmosferik (DOS) | Pathrow 124062 | Postfire
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-11-15
Persentase Cloud Cover: 23.63%


### Pathrow 125062 (Postfire)

In [13]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

start_date = '2023-11-01'
end_date = '2023-12-31'


landsat9_toa = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .first()


date_acquired = landsat9_toa.get('DATE_ACQUIRED').getInfo()
cloud_cover = landsat9_toa.get('CLOUD_COVER').getInfo()
print(f'Citra yang ditampilkan diambil pada tanggal: {date_acquired}')
print(f'Persentase Cloud Cover: {cloud_cover}%')


def dark_object_subtraction(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),  # mendekati nilai minimum
            geometry=band.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

dos_corrected = dark_object_subtraction(landsat9_toa, ['B7', 'B5', 'B4'])


false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}


m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'DOS False Color (B7-B5-B4)')
m.add_child(folium.LayerControl())


title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Koreksi Atmosferik (DOS) | Pathrow 125062 | Postfire
     </b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))
m


Citra yang ditampilkan diambil pada tanggal: 2023-11-06
Persentase Cloud Cover: 41.67%


## Cloud Masking

### Tanpa Tambal

In [14]:
#TANPA TAMBAL

import ee
import folium
import geemap.foliumap as geemap

# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# Pathrow dan rentang waktu data
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'

# Ambil koleksi citra LANDSAT 9 TOA
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

# Pilih citra dengan cloud cover terendah
image_best = collection.first()
roi_geometry = image_best.geometry()

# DARK OBJECT SUBTRACTION (DOS)
def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)

# Fungsi Cloud Masking menggunakan QA_PIXEL
def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')

    fill    = (1 << 0)
    dilated = (1 << 1)
    cirrus  = (1 << 2)
    cloud   = (1 << 3)
    shadow  = (1 << 4)
    snow    = (1 << 5)
    water   = (1 << 7)

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0)) \
        .And(qa.bitwiseAnd(snow).eq(0)) \
        .And(qa.bitwiseAnd(water).eq(0))

    return dos_img.updateMask(mask)

# Terapkan masking awan
masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()


false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}

m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(composite_after, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Cloud Masking | Pathrow 124062 | Prefire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


### Pathrow 124062 (Prefire)

In [29]:
import ee
import folium
import geemap.foliumap as geemap

#Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


#Pathrow dan rentang waktu data
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'

#Ambil koleksi citra LANDSAT 9 TOA berdasarkan Pathrow dan rentang waktu
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

#Pilih citra dengan cloud cover terendah
image_best = collection.first() #Mengambil citra pertama dari Collection berdasarkan pathrow, rentang waktu dan cloudcover
roi_geometry = image_best.geometry() #ROI geometri berdasarkan image_best, membantu DOS hanya spesifik di ROI

#DARK OBJECT SUBTRACTION (DOS)
def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

#Terapkan DOS pada setiap citra
def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

#Koleksi hasil koreksi atmosferik (DOS)
dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


# Fungsi Cloud Masking menggunakan QA_PIXEL dari citra asli
def cloudmask(dos_img):
    #Ambil system: index dari citra hasil DOS
    system_index = dos_img.get('system:index')
    #Ambil citra asli dari koleksi berdasarkan system:index
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()

    #Ambil band QA_PIXEL
    qa = original.select('QA_PIXEL')

    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan


    #Buat mask gabungan (piksel baik adalah yang tidak memiliki bit-bit tersebut)
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0)) \

        # Update mask pada citra hasil DOS
    return dos_img.updateMask(mask)

# Terapkan masking awan ke citra hasil DOS
masked_dos_collection = dos_collection.map(cloudmask)

# Composite setelah masking (akan ada lubang karena masking)
composite_after = masked_dos_collection.median()

#Tambal lubang dengan composite sebelum masking
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Cloud Masking | Pathrow 124062 | Prefire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


### Pathrow 125062 (Prefire)

In [30]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-01-01'
end_date = '2023-07-31'


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

image_best = collection.first()
roi_geometry = image_best.geometry()

def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')
    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return dos_img.updateMask(mask)

masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Cloud Masking | Pathrow 125062 | Prefire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


### Pathrow 124062 (Postfire)

In [17]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

image_best = collection.first()
roi_geometry = image_best.geometry()

def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')
    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return dos_img.updateMask(mask)

masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Cloud Masking | Pathrow 124062 | Postfire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


### Pathrow 125062 (Postfire)

In [18]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

image_best = collection.first()
roi_geometry = image_best.geometry()

def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')
    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return dos_img.updateMask(mask)

masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     Cloud Masking | Pathrow 125062 | Postfire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


## Komposit Citra

### False color 7-5-4 (Postfire) | 124

In [19]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

image_best = collection.first()
roi_geometry = image_best.geometry()

def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')
    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return dos_img.updateMask(mask)

masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     False Color | Pathrow 124062 | Postfire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


### False Color 7-5-4 (Postfire) | 125

In [20]:
import ee
import folium
import geemap.foliumap as geemap

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER')

image_best = collection.first()
roi_geometry = image_best.geometry()

def dos(img, band_names):
    def subtract_min(band_name):
        band = img.select(band_name)
        dark_object = band.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=roi_geometry,
            scale=30,
            maxPixels=1e9
        ).get(band_name)
        return band.subtract(ee.Number(dark_object)).rename(band_name)
    corrected_bands = [subtract_min(b) for b in band_names]
    return ee.Image.cat(corrected_bands).copyProperties(img, img.propertyNames())

def apply_dos(img):
    dos_img = dos(img, ['B7', 'B5', 'B4'])
    return dos_img.copyProperties(img, img.propertyNames())

dos_collection = collection.map(apply_dos)
composite_before = dos_collection.median()


def cloudmask(dos_img):
    system_index = dos_img.get('system:index')
    original = collection.filter(ee.Filter.eq('system:index', system_index)).first()
    qa = original.select('QA_PIXEL')
    #Bit untuk masking kualitas citra dari QA_PIXEL
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return dos_img.updateMask(mask)

masked_dos_collection = dos_collection.map(cloudmask)
composite_after = masked_dos_collection.median()
dos_corrected = composite_after.unmask(composite_before)

false_color_vis = {
    'bands': ['B7', 'B5', 'B4'],
    'min': 0.0,
    'max': 0.5,
}
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dos_corrected, false_color_vis, 'Cloud Masking (QA Bands)')
m.add_child(folium.LayerControl())

title_html = f'''
    <h3 align="center" style="font-size:20px"><b>
     False Color | Pathrow 125062 | Postfire
     </b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))
m


# Ekstraksi Fitur

## NDVI Pathrow 124062 (Postfire)

In [46]:
import ee
import folium
import geemap.foliumap as geemap

# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

#Filter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


# Fungsi DOS
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

#Fungsi Masking awan QA_PIXEL
def mask_fmask(image):
    qa = image.select('QA_PIXEL')

    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    return image.updateMask(mask)

# ========================================
# Ambil koleksi dan terapkan DOS
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)

# Hitung NDVI dari citra mentah (penambal)
ndvi_fill = collection.map(lambda img: img.normalizedDifference(['B5', 'B4']).rename('NDVI')).median()

# Masking awan
masked_collection = collection.map(mask_fmask)

# Hitung NDVI dari citra yang sudah di-mask
ndvi_masked = masked_collection.map(lambda img: img.normalizedDifference(['B5', 'B4']).rename('NDVI'))
ndvi_composite = ndvi_masked.median()

# Ambil geometri dari image pertama
first_image = collection.first()
geometry = first_image.geometry()

# Unmask bagian berlubang dan clip ke ROI
ndvi_filled = ndvi_composite.unmask(ndvi_fill).clip(geometry)

# ========================================
# Visualisasi NDVI
ndvi_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen']
}

# ========================================
# Tampilkan ke peta dengan FOLIUM
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(ndvi_filled, ndvi_vis_params, 'NDVI Pathrow 124062')

# Tambahkan Layer Control
m.add_child(folium.LayerControl())

# ========================================
# Tambahkan Legenda NDVI
legend_keys = [
    '< 0: Non Vegetasi',
    '0 - 0.2: Vegetasi Sangat Jarang',
    '0.2 - 0.4: Vegetasi Rendah',
    '0.4 - 0.6: Vegetasi Sedang-Lebat',
    '> 0.6: Vegetasi Sangat Lebat'
]

legend_colors = [
    (255, 0, 0),         # red
    (255, 165, 0),       # orange
    (255, 255, 0),       # yellow
    (0, 128, 0),         # green
    (0, 100, 0)          # darkgreen
]

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>NDVI pathrow 124062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# Tampilkan Peta
m


## NDVI Pathrow 125062 (Postfire)

In [48]:
import ee
import folium
import geemap.foliumap as geemap


def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer


path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'


def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)  # Bit 0: piksel kosong (invalid)
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    return image.updateMask(mask)


collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)

ndvi_fill = collection.map(lambda img: img.normalizedDifference(['B5', 'B4']).rename('NDVI')).median()

masked_collection = collection.map(mask_fmask)


ndvi_masked = masked_collection.map(lambda img: img.normalizedDifference(['B5', 'B4']).rename('NDVI'))
ndvi_composite = ndvi_masked.median()


first_image = collection.first()
geometry = first_image.geometry()
ndvi_filled = ndvi_composite.unmask(ndvi_fill).clip(geometry)


# Visualisasi NDVI
ndvi_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen']
}


# Tampilkan ke peta dengan FOLIUM
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(ndvi_filled, ndvi_vis_params, 'NDVI Pathrow 124062')
m.add_child(folium.LayerControl())

# Tambahkan Legenda NDVI
legend_keys = [
    '< 0: Non Vegetasi',
    '0 - 0.2: Vegetasi Sangat Jarang',
    '0.2 - 0.4: Vegetasi Rendah',
    '0.4 - 0.6: Vegetasi Sedang-Lebat',
    '> 0.6: Vegetasi Sangat Lebat'
]

legend_colors = [
    (255, 0, 0),         # red
    (255, 165, 0),       # orange
    (255, 255, 0),       # yellow
    (0, 128, 0),         # green
    (0, 100, 0)          # darkgreen
]

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>NDVI Pathrow 125062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))
m


## NBR Pathrow 124062 (Postfire)

In [50]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Filter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer Dark Object Subtraction (DOS1)
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan berdasarkan QA_PIXEL
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)  # Bit 0: piksel kosong
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    return image.updateMask(mask)

# =========================
# Ambil koleksi citra dan terapkan DOS
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)

# Hitung NBR dari citra mentah untuk penambal lubang
nbr_fill = collection.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()

# Masking awan
masked_collection = collection.map(mask_fmask)

# Hitung NBR dari citra yang sudah di-mask
nbr_masked = masked_collection.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
nbr_composite = nbr_masked.median()

# Ambil geometri dari citra pertama
first_image = collection.first()
geometry = first_image.geometry()

# Unmask bagian berlubang dan clip ke ROI
nbr_filled = nbr_composite.unmask(nbr_fill).clip(geometry)

# =========================
# Parameter visualisasi NBR
nbr_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen']
}

# =========================
# Tampilkan ke peta
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(nbr_filled, nbr_vis_params, 'NBR Pathrow 124062')
m.add_child(folium.LayerControl())

# Tambahkan Legenda NBR
legend_keys = [
    '< 0: Non Vegetasi',
    '0 - 0.2: Vegetasi Sangat Jarang',
    '0.2 - 0.4: Vegetasi Rendah',
    '0.4 - 0.6: Vegetasi Sedang-Lebat',
    '> 0.6: Vegetasi Sangat Lebat'
]

legend_colors = [
    (255, 0, 0),         # red
    (255, 165, 0),       # orange
    (255, 255, 0),       # yellow
    (0, 128, 0),         # green
    (0, 100, 0)          # darkgreen
]


legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>NBR Pathrow 124062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# =========================
# Tampilkan peta interaktif
m


## NBR Pathrow 125062 (Postfire)

In [51]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Filter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)
start_date = '2023-11-01'
end_date = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer Dark Object Subtraction (DOS1)
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan berdasarkan QA_PIXEL
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)  # Bit 0: piksel kosong
    dilated = (1 << 1)  # Bit 1: buffer awan
    cirrus  = (1 << 2)  # Bit 2: awan tipis
    cloud   = (1 << 3)  # Bit 3: awan
    shadow  = (1 << 4)  # Bit 4: bayangan awan

    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))

    return image.updateMask(mask)

# =========================
# Ambil koleksi citra dan terapkan DOS
collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(start_date, end_date) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)

# Hitung NBR dari citra mentah untuk penambal lubang
nbr_fill = collection.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()

# Masking awan
masked_collection = collection.map(mask_fmask)

# Hitung NBR dari citra yang sudah di-mask
nbr_masked = masked_collection.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
nbr_composite = nbr_masked.median()

# Ambil geometri dari citra pertama
first_image = collection.first()
geometry = first_image.geometry()

# Unmask bagian berlubang dan clip ke ROI
nbr_filled = nbr_composite.unmask(nbr_fill).clip(geometry)

# =========================
# Parameter visualisasi NBR
nbr_vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen']
}

# =========================
# Tampilkan ke peta
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(nbr_filled, nbr_vis_params, 'NBR Pathrow 124062')
m.add_child(folium.LayerControl())

# Tambahkan Legenda NBR
legend_keys = [
    '< 0: Non Vegetasi',
    '0 - 0.2: Vegetasi Sangat Jarang',
    '0.2 - 0.4: Vegetasi Rendah',
    '0.4 - 0.6: Vegetasi Sedang-Lebat',
    '> 0.6: Vegetasi Sangat Lebat'
]

legend_colors = [
    (255, 0, 0),         # red
    (255, 165, 0),       # orange
    (255, 255, 0),       # yellow
    (0, 128, 0),         # green
    (0, 100, 0)          # darkgreen
]


legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>NBR Pathrow 125062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# =========================
# Tampilkan peta interaktif
m


## DNBR

### 124062

In [72]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Parameter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

# Prefire dan postfire
prefire_start = '2023-01-01'
prefire_end   = '2023-07-31'
postfire_start = '2023-11-01'
postfire_end   = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer Dark Object Subtraction (DOS1)
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)
    dilated = (1 << 1)
    cirrus  = (1 << 2)
    cloud   = (1 << 3)
    shadow  = (1 << 4)
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return image.updateMask(mask)

# =========================
# Ambil koleksi prefire
prefire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(prefire_start, prefire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
prefire_fill = prefire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
prefire_masked = prefire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
prefire_nbr = prefire_masked.median().unmask(prefire_fill)

# Ambil koleksi postfire
postfire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(postfire_start, postfire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
postfire_fill = postfire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
postfire_masked = postfire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
postfire_nbr = postfire_masked.median().unmask(postfire_fill)

# Hitung dNBR
dnbr = prefire_nbr.subtract(postfire_nbr).rename('dNBR')

# =========================
# Visualisasi dNBR
dnbr_vis_params = {
    'min': -0.2,
    'max': 1,
    'palette': ['green', 'yellow', 'orange', 'red', 'black']
}

# =========================
# Peta
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dnbr, dnbr_vis_params, 'dNBR')
m.add_child(folium.LayerControl())

# =========================
# Legenda dNBR (4 kelas)
legend_keys = [
    '< 0.1: Tidak Terbakar',
    '0.1 - 0.27: Keparahan Rendah',
    '0.1 - 0.27: Keparahan Rendah-Sedang',
    '0.27 - 0.66: Keparahan Sedang-Tinggi',
    '> 0.66: Keparahan Tinggi'
]

legend_colors = [
    (0, 100, 0),       # darkgreen
    (0, 128, 0),       # darkgreen
    (255, 255, 0),     # yellow
    (255, 165, 0),     # orange
    (255, 0, 0),       # red
]

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>dNBR Pathrow 124062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# =========================
# Tampilkan
m


### 125062

In [70]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Parameter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

# Prefire dan postfire
prefire_start = '2023-01-01'
prefire_end   = '2023-07-31'
postfire_start = '2023-11-01'
postfire_end   = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer Dark Object Subtraction (DOS1)
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)
    dilated = (1 << 1)
    cirrus  = (1 << 2)
    cloud   = (1 << 3)
    shadow  = (1 << 4)
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return image.updateMask(mask)

# =========================
# Ambil koleksi prefire
prefire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(prefire_start, prefire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
prefire_fill = prefire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
prefire_masked = prefire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
prefire_nbr = prefire_masked.median().unmask(prefire_fill)

# Ambil koleksi postfire
postfire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(postfire_start, postfire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
postfire_fill = postfire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
postfire_masked = postfire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
postfire_nbr = postfire_masked.median().unmask(postfire_fill)

# Hitung dNBR
dnbr = prefire_nbr.subtract(postfire_nbr).rename('dNBR')

# =========================
# Visualisasi dNBR
dnbr_vis_params = {
    'min': -0.2,
    'max': 1,
    'palette': ['green', 'yellow', 'orange', 'red', 'black']
}

# =========================
# Peta
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dnbr, dnbr_vis_params, 'dNBR')
m.add_child(folium.LayerControl())

# =========================
# Legenda dNBR (4 kelas)
legend_keys = [
    '< 0.1: Tidak Terbakar',
    '0.1 - 0.27: Keparahan Rendah',
    '0.1 - 0.27: Keparahan Rendah-Sedang',
    '0.27 - 0.66: Keparahan Sedang-Tinggi',
    '> 0.66: Keparahan Tinggi'
]

legend_colors = [
    (0, 100, 0),       # darkgreen
    (0, 128, 0),       # darkgreen
    (255, 255, 0),     # yellow
    (255, 165, 0),     # orange
    (255, 0, 0),       # red
]

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>dNBR Pathrow 125062</b><br>
'''

for label, color in zip(legend_keys, legend_colors):
    rgb = f'rgb{color}'
    legend_html += f'<i style="background:{rgb};width:12px;height:12px;display:inline-block;margin-right:6px;"></i>{label}<br>'

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# =========================
# Tampilkan
m


## Masking Citra

### 124062

In [90]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Parameter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 124)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

# Prefire dan postfire
prefire_start = '2023-01-01'
prefire_end   = '2023-07-31'
postfire_start = '2023-11-01'
postfire_end   = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer DOS1
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)
    dilated = (1 << 1)
    cirrus  = (1 << 2)
    cloud   = (1 << 3)
    shadow  = (1 << 4)
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return image.updateMask(mask)

# =========================
# Koleksi prefire
prefire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(prefire_start, prefire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
prefire_fill = prefire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
prefire_masked = prefire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
prefire_nbr = prefire_masked.median().unmask(prefire_fill)

# Koleksi postfire
postfire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(postfire_start, postfire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
postfire_fill = postfire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
postfire_masked = postfire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
postfire_nbr = postfire_masked.median().unmask(postfire_fill)

# Hitung dNBR
dnbr = prefire_nbr.subtract(postfire_nbr).rename('dNBR')

# Thresholding: dNBR > 0.5
dnbr_mask = dnbr.gt(0.5).rename('BurnedMask').uint8()


# Visualisasi (putih = terbakar, hitam = tidak terbakar)
mask_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'ffffff']
}

# =========================
# Peta dan tampilan
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dnbr_mask, mask_vis_params, 'Burned Area Mask (dNBR > 0.1)')
m.add_child(folium.LayerControl())

# =========================
# Legenda
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>Citra Mask 124062</b><br>
<i style="background:rgb(255,255,255);width:12px;height:12px;display:inline-block;margin-right:6px;"></i>Terbakar<br>
<i style="background:rgb(0,0,0);width:12px;height:12px;display:inline-block;margin-right:6px;"></i>Tidak Terbakar<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Tampilkan
m


### 125062

In [85]:
import ee
import folium
import geemap.foliumap as geemap

# =========================
# Fungsi agar folium bisa menampilkan EE layer
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

# =========================
# Parameter lokasi dan waktu
path_filter = ee.Filter.eq('WRS_PATH', 125)
row_filter = ee.Filter.eq('WRS_ROW', 62)
pathrow_filter = ee.Filter.And(path_filter, row_filter)

# Prefire dan postfire
prefire_start = '2023-01-01'
prefire_end   = '2023-07-31'
postfire_start = '2023-11-01'
postfire_end   = '2023-12-31'

# =========================
# Fungsi koreksi atmosfer DOS1
def apply_dos1(image):
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
    corrected = []
    for band in bands:
        band_data = image.select(band)
        dark_object = band_data.reduceRegion(
            reducer=ee.Reducer.percentile([1]),
            geometry=image.geometry(),
            scale=30,
            maxPixels=1e9
        ).getNumber(band)
        corrected_band = band_data.subtract(dark_object).max(0).rename(band)
        corrected.append(corrected_band)
    return image.addBands(ee.Image.cat(corrected), overwrite=True)

# =========================
# Fungsi masking awan
def mask_fmask(image):
    qa = image.select('QA_PIXEL')
    fill    = (1 << 0)
    dilated = (1 << 1)
    cirrus  = (1 << 2)
    cloud   = (1 << 3)
    shadow  = (1 << 4)
    mask = qa.bitwiseAnd(fill).eq(0) \
        .And(qa.bitwiseAnd(dilated).eq(0)) \
        .And(qa.bitwiseAnd(cirrus).eq(0)) \
        .And(qa.bitwiseAnd(cloud).eq(0)) \
        .And(qa.bitwiseAnd(shadow).eq(0))
    return image.updateMask(mask)

# =========================
# Koleksi prefire
prefire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(prefire_start, prefire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
prefire_fill = prefire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
prefire_masked = prefire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
prefire_nbr = prefire_masked.median().unmask(prefire_fill)

# Koleksi postfire
postfire = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
    .filter(pathrow_filter) \
    .filterDate(postfire_start, postfire_end) \
    .sort('CLOUD_COVER') \
    .map(apply_dos1)
postfire_fill = postfire.map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR')).median()
postfire_masked = postfire.map(mask_fmask).map(lambda img: img.normalizedDifference(['B5', 'B7']).rename('NBR'))
postfire_nbr = postfire_masked.median().unmask(postfire_fill)

# Hitung dNBR
dnbr = prefire_nbr.subtract(postfire_nbr).rename('dNBR')

# Thresholding: dNBR > 0.5
dnbr_mask = dnbr.gt(0.5).rename('BurnedMask').uint8()


# Visualisasi (putih = terbakar, hitam = tidak terbakar)
mask_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'ffffff']
}

# =========================
# Peta dan tampilan
m = folium.Map(location=[-3.3194, 104.7458], zoom_start=8)
m.add_ee_layer(dnbr_mask, mask_vis_params, 'Burned Area Mask (dNBR > 0.1)')
m.add_child(folium.LayerControl())

# =========================
# Legenda
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 10px;
    z-index: 9999;
    background-color: white;
    padding: 10px;
    border: 2px solid grey;
    border-radius: 5px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size:14px;
">
<b>Citra Mask 125062</b><br>
<i style="background:rgb(255,255,255);width:12px;height:12px;display:inline-block;margin-right:6px;"></i>Terbakar<br>
<i style="background:rgb(0,0,0);width:12px;height:12px;display:inline-block;margin-right:6px;"></i>Tidak Terbakar<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Tampilkan
m
